In [5]:
# Adapt the code as required for your business problem
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from math import sqrt
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.linear_model import LinearRegression, Lasso, RidgeCV, LassoCV, Ridge, ElasticNetCV, ElasticNet
from sklearn.feature_selection import SelectKBest, SelectPercentile, f_regression
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split, cross_val_score, LeaveOneOut, GridSearchCV
from sklearn.tree import DecisionTreeRegressor, export_graphviz
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler, scale
from sklearn.feature_selection import VarianceThreshold, RFE, SelectFromModel
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
import pylab
from sklearn import metrics
import warnings
warnings.filterwarnings("ignore")
pd.pandas.set_option('display.max_columns', None)

In [ ]:
# load prepared dataset (after performig necessary feature engineering for: numeric, categorial, temporal and discrete variables)
# data = pd.read_csv('Sales or price.csv')
# print(data.shape)
# data.head()

# split Train and test set if not done before
# X_train, X_test, y_train, y_test = train_test_split(data, data.SalePrice,
#                                                    test_size=0.2,
#                                                    random_state=0) # we are setting the seed here

In [ ]:
# Load train and test dataset if ready
X_train = pd.read_csv('train.csv')
X_test = pd.read_csv('test.csv')

print(X_train.head())
print('Dataframe shape: ', X_train.shape)
X_train.describe()

In [ ]:
# set the target
y_train = X_train['Sales or SalePrice']
y_test = X_test['Sales or SalePrice']

# drop unnecessary variables from our train and test sets: add columns
X_train.drop(['Id', 'Sales or Price'], axis=1, inplace=True)
X_test.drop(['Id', 'Sales or Price'], axis=1, inplace=True)

In [ ]:
def feature_selection_pipeline(X_train, X_test):
    ### remove constant features
    constant_features = [feat for feat in X_train.columns if X_train[feat].std() == 0]

    X_train.drop(labels=constant_features, axis=1, inplace=True)
    X_test.drop(labels=constant_features, axis=1, inplace=True)

    # Print shape after removing constant features
    print(X_train.shape, X_test.shape)
    
### remove quasi-constant features
    sel = VarianceThreshold(
        threshold=0.01)  # 0.1 indicates 99% of observations approximately

    sel.fit(X_train)  # fit finds the features with low variance

    sum(sel.get_support()) # how many not quasi-constant?

    features_to_keep = X_train.columns[sel.get_support()]

    # we can then remove the features like this
    X_train = sel.transform(X_train)
    X_test = sel.transform(X_test)
    # Print shape after removing quasi-constant features
    print(X_train.shape, X_test.shape)

    # sklearn transformations lead to numpy arrays
    # transform the arrays back to dataframes & getting the columns assigned correctly

    X_train= pd.DataFrame(X_train)
    X_train.columns = features_to_keep

    X_test= pd.DataFrame(X_test)
    X_test.columns = features_to_keep
   
    ### check for duplicated features in the training set
    duplicated_feat = []
    for i in range(0, len(X_train.columns)):
        if i % 10 == 0:  #  understand how the loop is going
            #print(i)
            col_1 = X_train.columns[i]

        for col_2 in X_train.columns[i + 1:]:
            if X_train[col_1].equals(X_train[col_2]):
                duplicated_feat.append(col_2)
                
    len(duplicated_feat)

    # remove duplicated features
    X_train.drop(labels=duplicated_feat, axis=1, inplace=True)
    X_test.drop(labels=duplicated_feat, axis=1, inplace=True)

    # Print shape after removing duplicated features
    print(X_train.shape, X_test.shape)

    ### find and remove correlated features
    def correlation(dataset, threshold):
        col_corr = set()  # Set names of correlated columns
        corr_matrix = dataset.corr()
        for i in range(len(corr_matrix.columns)):
            for j in range(i):
                if abs(corr_matrix.iloc[i, j]) > threshold: # absolute coeff value
                    colname = corr_matrix.columns[i]  
                    col_corr.add(colname)
        return col_corr

    corr_features = correlation(X_train, 0.95)
    print('correlated features: ', len(set(corr_features)) )

    X_train.drop(labels=corr_features, axis=1, inplace=True)
    X_test.drop(labels=corr_features, axis=1, inplace=True)

    # Print shape after removing remove correlated features
    print(X_train.shape, X_test.shape)
    
    return X_train, X_test

X_train, X_test = feature_selection_pipeline(X_train, X_test)


# model fitting and feature selection with Lasso Regression
# select suitable alpha (equivalent of penalty).
# Bigger the alpha the less features will be selected.

#  selectFromModel object from sklearn to select the features which coefficients are non-zero

sel_ = SelectFromModel(Lasso(alpha=0.005, random_state=0)) # set the seed
sel_.fit(X_train, y_train)

# identifying the selected features 
selected_feats = X_train.columns[(sel_.estimator_.coef_ != 0).ravel().tolist()]

#  save the selected list of features
pd.Series(selected_feats).to_csv('selected_features.csv', index=False)

In [ ]:
# load selected features
features = pd.read_csv('selected_features.csv', header=None)
features = [x for x in features[0]] 

# reduce train and test set to selected features

X_train = X_train[features]
X_test = X_test[features]

In [ ]:
# Regularized linear model - train the model
lin_model = Lasso(alpha=0.005, random_state=0) # set random_state / seed
lin_model.fit(X_train, y_train)


# evaluate the model:
pred = lin_model.predict(X_train)
print('linear train mse: {}'.format(mean_squared_error(np.exp(y_train), np.exp(pred))))
print('linear train rmse: {}'.format(sqrt(mean_squared_error(np.exp(y_train), np.exp(pred)))))
print() # blank line
pred = lin_model.predict(X_test)
print('linear test mse: {}'.format(mean_squared_error(np.exp(y_test), np.exp(pred))))
print('linear test rmse: {}'.format(sqrt(mean_squared_error(np.exp(y_test), np.exp(pred)))))
print() # blank line
print('Average price or sales: ', np.exp(y_train).median())

In [ ]:
# evaluate our predictions respect to the original price or sales
plt.scatter(y_test, lin_model.predict(X_test))
plt.xlabel('True Price or Sales')
plt.ylabel('Predicted Price or Sales')
plt.title('Evaluation of Lasso Predictions')

In [ ]:
# evaluate the distribution of the errors: should be fairly normally distributed

errors = y_test - lin_model.predict(X_test)
errors.hist(bins=15)